# Graded Assignment 3: 9 to 5

Time to show off your SQL skills! For each question, copy the SQL query you used and make note of the answer.

## The Dataset

For this assignment, you will be using the Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results which can be found on [Kaggle](https://www.kaggle.com/datasets/bls/employment).

## Business Issue

You are working for the Bureau of Labor Statistics with the United States government and have been approached by your boss with an important meeting request. You have been asked by your supervisor to meet with Dolly Parton whose nonprofit is looking to shed light on the state of employment in the United States. As part of the 9 to 5 project, their research is focused on production and nonsupervisory employees and how those employees fare compared to all employees in the United States. While the data the BLS collects from the CES is publicly available, Dolly Parton and her colleagues need your assistance navigating the thousands of rows in each table in LaborStatisticsDB.

## About the Dataset

This dataset comes directly from the Bureau of Labor Statistics’ Current Employment Survey (CES). Here are some things you need to know:

1. The industry table contains an NAICS code. This is different from the industry code. NAICS stands for North American Industry Classification System.
1. Series ID is composed of multiple different codes. CES stands for Current Employment Survey, the name of the survey which collected the data. The industry code as specified by the BLS and the data type code as specified in the datatype table.

## Set Up

To connect to the database, use the same connection info used during the SQL lessons. 

For the assignment, we will be using `LaborStatisticsDB`.

## Database Exploration

To start with, let’s get to know the database further.

1. Use this space to make note of each table in the database, the columns within each table, each column’s data type, and how the tables are connected. You can write this down or draw a diagram. Whatever method helps you get an understanding of what is going on with `LaborStatisticsDB`.
   
   To add a photo, diagram or document to your file, drop the file into the folder that holds this notebook.  Use the link button to the right of the  </> symbol in the gray part of this cell, the link is just the name of your file.

![ER Diagram](./ER_Diagram.png)

LaborStatisticsDB Table Relationships

In [ ]:
--This dataset comes from the Bureau of Labor Statistics and contains employment data for 2016 and 2017 in the United States.
--There are nine total table contained in LaborStatisticsDB. The relationships between these tables are modeled in the above diagram. 
--Each table has a primary key, shown in bold and with a key symbol. Foreign keys within the tables are represented with the chain link symbol.
--The series table is the most central table for this dataset, linking to six other tables within this dataset as follows:
            --series connect to tables annual_2016 and january_2017 through the key series_id. 
            --series connects to the datatype table through the key data_type_code.
            --series connects to the industry table through the key industry_code.
            --series connects to the supersector table through the key supersector_code.
            --series references the seasonal table through the key seasonal_code to determine whether data is seasonally adjusted.

--The period table connects to tables annual_2016 and january_2017 through the key period_code via the period columns. This can be used to group data by month of the year.

2. What is the datatype for women employees?

In [ ]:
--The datatype code for women employees can be viewed with a quick scan of the Series table. However, to accomplish this with a query: 

Select series_title, data_type_code
FROM dbo.series
Where series_title = 'Women employees'

--Answer: The data_type_code for women employees is 10.

3. What is the series id for  women employees in the commercial banking industry in the financial activities supersector?

In [ ]:
--From inspecting the series table, it appears that 
--series_id is equal to CE+[seasonal code]+[industry code]+[data_type_code]
--In order to return the correct series_id, we can run this query:

SELECT DISTINCT *
FROM dbo.series as se
JOIN dbo.supersector as su
    ON se.supersector_code = su.supersector_code
JOIN dbo.industry as i
    ON se.industry_code = i.industry_code
WHERE su.supersector_name = 'financial activities'
    AND i.industry_name = 'commercial banking'
    AND se.data_type_code = 10;

--There are two series ids for this question. One seasonally adjusted (S), one not seasonally adjusted (U).
--Answer: CES5552211010 and CEU5552211010


## Aggregate Your Friends and Code some SQL

Put together the following:

1. How many employees were reported in 2016 in all industries? Round to the nearest whole number.

In [ ]:
--As shown the datatype table, data_type_code 1 filters for all employees.
--As shown in the supersector table, supersector_code 0 represents all nonfarm employees. Filtering on this reduces industry duplicates (eg, 'mining and logging', 'mining', and 'logging').
--annual_2016 also contains duplicates of every row as this data table pulled data from both "ce.data.0.AllCESSeries.csv" and "ce.data.0.AllCESSeries.csv". This is de-duplicated using a subquery on year and period.
--Note: Final value is multiplied by 1000 to account for how BLS annual values are reported

SELECT ROUND((SUM(value)*1000), 0) AS total_employees
FROM (SELECT DISTINCT series_id, value
    FROM dbo.annual_2016
    WHERE year = 2016
    AND period = 'M13') AS a
JOIN dbo.series as s
    ON a.series_id = s.series_id
WHERE s.data_type_code = 1
    AND s.supersector_code = 0

--Answer: 
--Total nonfarm employment for 2016 (not seasonally adjusted): 144,306,000 employees

2. How many women employees were reported in 2016 in all industries? Round to the nearest whole number. 

In [ ]:
--This will be the same query as above, with the data_type_code adjusted to '10', which we know from question 2 above.

SELECT ROUND((SUM(value)*1000), 0) AS total_women_employees
FROM (SELECT DISTINCT series_id, value
    FROM dbo.annual_2016
    WHERE year = 2016
    AND period = 'M13') AS a
JOIN dbo.series as s
    ON a.series_id = s.series_id
WHERE s.data_type_code = 10
    AND s.supersector_code = 0

--Answer:
--Total nonfarm women employment for 2016 (not seasonally adjusted): 71,492,000 women employees


3. How many production/nonsupervisory employees were reported in 2016? Round to the nearest whole number. 

In [ ]:
--Using the same query with the data_type_code adjusted to '6' for 'production and nonsupervisory roles' gives a NULL output.
--BLS does not report production/nonsupervisory employees in the overall report as these positions are not counted in government roles. 
--Change supersector_code to private industry only ('5') to get an accurate total.

SELECT ROUND((SUM(value)*1000), 0) AS total_women_employees
FROM (SELECT DISTINCT series_id, value
    FROM dbo.annual_2016
    WHERE year = 2016
    AND period = 'M13') AS a
JOIN dbo.series as s
    ON a.series_id = s.series_id
WHERE s.data_type_code = 6
    AND s.supersector_code = 5

--Answer:
--Total production and nonsupervisory employment for 2016 (not seasonally adjusted): 100,531,000 employees



4. In January 2017, what is the average weekly hours worked by production and nonsupervisory employees across all industries?

In [ ]:
--Use the previous query as a starting point
--Replace annual_2016 table with annual_2017 table, update de-duplicating conditions to year = 2017, period = M01
--Change data_type_code to 7 for average weekly hours of production and nonsupervisory employees
--Remove the SUM since the value presented already reprsents an average

SELECT value, series_title, seasonal
FROM (SELECT DISTINCT series_id, value
    FROM dbo.january_2017
    WHERE year = 2017
    AND period = 'M01') AS j
JOIN dbo.series as s
    ON j.series_id = s.series_id
WHERE s.data_type_code = 7
    AND s.supersector_code = 5

--Answer:
--33.5 average weekly hours worked by production and nonsupervisory employess across all industries (Not Seasonally Adjusted)
--33.6 average weekly hours worked by production and nonsupervisory employess across all industries (Seasonally adjusted)

5. What is the total weekly payroll for production and nonsupervisory employees across all industries in January 2017? Round to the nearest penny.

In [ ]:
--Use the previous query as a starting point
--Change data_type_code to 82, which is an aggregate of weekly payroll for production and nonspuervisory employees across all industries.
--Round and cast the value, multiply by 1000 to account for BLS reporting in thousands

SELECT CAST(ROUND(value * 1000, 2) AS DECIMAL(20,2)) AS total_payroll, series_title, seasonal
FROM (SELECT DISTINCT series_id, value
    FROM dbo.january_2017
    WHERE year = 2017
    AND period = 'M01') AS j
JOIN dbo.series as s
    ON j.series_id = s.series_id
WHERE s.data_type_code = 82
    AND s.supersector_code = 5

--Answer:
--$73,474,302,000 total weekly payroll of production and nonsupervisory employess across all industries (Not Seasonally Adjusted)
--$74,498,171,000 total weekly payroll of production and nonsupervisory employess across all industries (Seasonally adjusted)

6. In January 2017, for which industry was the average weekly hours worked by production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
--Can be achieved using the previous query as a starting point, and joining industry table on industry code
--Filter join on data_type_code 7 for average weekly hours of production and nonsupervisory employees

SELECT TOP 1 value, industry_name
FROM (SELECT DISTINCT series_id, value
    FROM dbo.january_2017
    WHERE year = 2017
    AND period = 'M01') AS j
JOIN dbo.series as s
    ON j.series_id = s.series_id
JOIN dbo.industry as i
    ON i.industry_code = s.industry_code
    WHERE s.data_type_code = 7
ORDER BY value DESC;

--For lowest, use same query, swap out descending order for ascending

SELECT TOP 1 value, industry_name
FROM (SELECT DISTINCT series_id, value
    FROM dbo.january_2017
    WHERE year = 2017
    AND period = 'M01') AS j
JOIN dbo.series as s
    ON j.series_id = s.series_id
JOIN dbo.industry as i
    ON i.industry_code = s.industry_code
    WHERE s.data_type_code = 7
ORDER BY value ASC;

--Answers:
--For January 2017, the highest average weekly hours worked by production and nonsupervisory employees was in the Motor vehicle power train components industry, at 49.8 hours, seasonally adjusted. (Phheww, that's a hard-earned weekend)
--For January 2017, the lowest average weekly hours worked by production and nonsupervisory employees was in the Fitness and recreational sport centers industry, at 16.7 hours, not seasonally adjusted.

7. In January 2017, for which industry was the total weekly payroll for production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
--Can be achieved using the previous query as a starting point, and swapping out data_type_code 7 for 82 (aggregate weekly payroll of production and nonsupervisory employees)
--The initial outputs for this query were too broad ('Total private', 'Professional and Business Services', etc). Add a filter for 'display level'.
--display_level ranges from 1-7, if further granularity is desired, increase display level cut-off in the query. I used display_level 4, as this is the midpoint for the NAICS range.

SELECT industry_name, (value * 1000) AS total_weekly_payroll
FROM (SELECT DISTINCT series_id, value
    FROM dbo.january_2017
    WHERE year = 2017
    AND period = 'M01') AS j
JOIN dbo.series as s
    ON j.series_id = s.series_id
JOIN dbo.industry as i
    ON i.industry_code = s.industry_code
    WHERE s.data_type_code = 82
    AND i.display_level >= 4
ORDER BY value DESC;

--For lowest, use same query, swap out descending order for ascending

SELECT industry_name, (value * 1000) AS total_weekly_payroll
FROM (SELECT DISTINCT series_id, value
    FROM dbo.january_2017
    WHERE year = 2017
    AND period = 'M01') AS j
JOIN dbo.series as s
    ON j.series_id = s.series_id
JOIN dbo.industry as i
    ON i.industry_code = s.industry_code
    WHERE s.data_type_code = 82
    AND i.display_level >= 4
ORDER BY value ASC;

--Answers:
--For January 2017, the highest total weekly payroll for production and nonsupervisory employees was in the industry categorized as 'Hospitals', at $5,105,336,000, not seasonally adjusted.
--For January 2017, the lowest total weekly payroll for production and nonsupervisory employees was in the industry categorized as 'Coin-operated laundries and drycleaners', at $10,079,000, not seasonally adjusted.


## Join in on the Fun

Time to start joining! You can choose the type of join you use, just make sure to make a  note!

1. Join `annual_2016` with `series` on `series_id`. We only want the data in the `annual_2016` table to be included in the result.

In [ ]:
SELECT TOP 50 
    a.id, a.series_id, a.year, a.period, a.VALUE
    s.series_title, s.data_type_code, s.industry_code, s.seasonal
FROM dbo.annual_2016 AS a
LEFT JOIN dbo.series AS s
    ON a.series_id = s.series_id
ORDER BY a.id

--LEFT JOIN to keep only annual_2016 data
--Specified ORDER BY id as from a.id

Commands completed successfully.

Total execution time: 00:00:00.019

2. Join `series` and `datatype` on `data_type_code`.

In [ ]:
SELECT TOP 50
    s.series_id, s.data_type_code, d.data_type_text
FROM dbo.series AS s
INNER OIN dbo.datatype AS d
    ON d.data_type_code = s.data_type_code
ORDER BY s.series_id

--INNER JOIN since the datatype table is a lookup table (adding definitions to the series data)
--Changed 'ORDER BY id' to 'ORDER BY s.series_id' as there is no 'id' column in either of the tables we are joining here

3. Join `series` and `industry` on `industry_code`.

In [ ]:
SELECT TOP 50
    s.series_id, s.industry_code, i.industry_name, i.naics_code
FROM dbo.series AS s
INNER JOIN dbo.industry AS i
    ON i.industry_code = s.industry_code
ORDER BY i.id

--INNER JOIN since the industry table is a lookup table (adding definitions to the series data)
--Specified ORDER BY id as from i.id

## Subqueries, Unions, Derived Tables, Oh My!

1. Write a query that returns the `series_id`, `industry_code`, `industry_name`, and `value` from the `january_2017` table but only if that value is greater than the average value for `annual_2016` of `data_type_code` 82.

In [ ]:
--data_type_code 82 is aggregate weekly payrolls of production and nonsupervisory rolls. 
--First, create a subquery to determine the average weekly payroll across all industries for production and nonsupervisory staff across the year 2016.

SELECT AVG(VALUE) AS avg_2016_weekly_payroll
FROM dbo.annual_2016 AS a
JOIN dbo.series AS s
ON a.series_id = s.series_id
WHERE data_type_code = 82
AND s.seasonal = 'U';

--Average 2016 weekly payroll (not seasonally adjusted) for all production and nonsupervisory staff is $819,960
--Now, use this subquery to filter out industries with average weekly payrolls from january_2017 lower than the average 2016 value determined above.


SELECT DISTINCT
    j.series_id, s.industry_code, i.industry_name, j.value AS jan_2017_weekly_payrolls
FROM dbo.january_2017 AS j
JOIN dbo.series AS s
    ON j.series_id = s.series_id
JOIN dbo.industry AS i
    ON s.industry_code = i.industry_code
WHERE s.data_type_code = 82
AND s.seasonal = 'U'
AND j.value > (
SELECT AVG(a.VALUE) AS avg_2016_weekly_payroll
FROM dbo.annual_2016 AS a
JOIN dbo.series AS s2
    ON a.series_id = s2.series_id
WHERE s2.data_type_code = 82
)

--This query outputs all industries where production and nonsupervisory staff had average weekly payrolls (not seasonally adjusted) in January 2017 that were higher than the average weekly payroll in 2016. 
--This represents industries that were thriving in the first month of 2017 compared to 2016, as represented by average weekly payrolls.

**Optional Bonus Question:** Write the above query as a common table expression!

In [ ]:
--Use the subquery developed earlier as a common table expression

WITH weekly_payroll_cte AS (
SELECT AVG(VALUE) AS avg_weekly_payroll
FROM dbo.annual_2016 AS a
JOIN dbo.series AS s
ON a.series_id = s.series_id
WHERE data_type_code = 82
)
SELECT
    j.series_id, s.industry_code, i.industry_name, j.value AS jan_2017_weekly_payrolls
FROM dbo.january_2017 AS j
JOIN dbo.series AS s
    ON j.series_id = s.series_id
JOIN dbo.industry AS i
    ON s.industry_code = i.industry_code
WHERE s.data_type_code = 82
AND j.value > (SELECT avg_weekly_payroll FROM weekly_payroll_cte)

--This outputs an identical result to the subquery version from part 1.

2. Create a `Union` table comparing average weekly earnings of production and nonsupervisory employees between `annual_2016` and `january_2017` using the data type 30.  Round to the nearest penny.  You should have a column for the average earnings and a column for the year, and the period.

In [ ]:
--data_type_code 30 represents average weekly earnings of production and nonsupervisory employees
--Join the annual_2016 table with the series table, and join the january_2017 table with the series table, then union the outputs after filtering for data_type_code 30

SELECT CAST(ROUND(a.value, 2) AS DECIMAL(12,2)) AS average_earnings, a.year, a.period
FROM dbo.annual_2016 AS a
JOIN dbo.series AS s
    ON a.series_id = s.series_id
WHERE s.data_type_code = 30

UNION

SELECT CAST(ROUND(j.value, 2) AS DECIMAL(12,2)) AS average_earnings, j.year, j.period
FROM dbo.january_2017 AS j
JOIN dbo.series AS s
    ON j.series_id = s.series_id
WHERE s.data_type_code = 30

--This generates a table that includes average weekly earnings of production and nonsupervisory employees for all industries in both 2016 and January 2017.

## Summarize Your Results

With what you know now about the  Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results and working with the Labor Statistics Database, answer the following questions. Note that while this is subjective, you should include relevant data to back up your opinion.

1. During which time period did production and nonsupervisory employees fare better?

To compare average weekly payrolls during the two time periods of January 2017 and 2016, I begain with the same query structure from Subqueries Part 1, and re-formatted it to calculate the average for January 2017 the same way we calculated for 2016. This query is shown below.

This query reveals that the January 2017 average weekly payroll was \$829,074. So, even though we found specific industries that saw payroll increases from 2016 to January 2017...
we can conclude that production and nonsupervisory employees across all industries fared better in 2016 than they did in January 2017, to the tune of a \$9,100 average weekly payroll decrease from 2016 to January 2017.


In [ ]:
SELECT AVG(VALUE) AS jan_2017_weekly_payroll
FROM dbo.january_2017 AS j
JOIN dbo.series AS s
ON j.series_id = s.series_id
WHERE data_type_code = 82
AND s.seasonal = 'U'
----Average January 2017 weekly payroll (not seasonally adjusted) for all production and nonsupervisory staff is $829,074

2. In which industries did production and nonsupervisory employees fare better?

In order to determine which industries fared better, I re-ran the earlier query from Subqueries Part 1 with an additional ORDER BY clause to sort by descending value. Then, I targeted industries that were within the same order of magnitude as the previously computed averages (\$820k-\$830k) to confirm we are comparing industries within similar NCAIS classification levels.

A few industries with production and nonsupervisory employees that fared better than average include Grocery stores (\$861,224 weekly payroll), Real estate (\$865,620 weekly payroll), and Food manufacturing (\$842,201 weekly payroll).

It's worth noting that this analysis compares industry-specific weekly payroll averages against the industry-wide average. It might be more useful for future analyses to compare each industry's weekly payroll against those same industries' weekly payrolls from previous time periods, rather than using an average.

3. Now that you have explored the datasets, is there any data or information that you wish you had in this analysis?

It would be nice to have datasets covering a wider range of time periods. A full dataset of the past 5-10 years would allow filtering by month using the 'period' column, which would enable me to determine which time periods have seen the strongest and weakest performance from the perpsective of average weekly payrolls. I would also like more information from the BLS detailing the industry hierarchies, as there is a lack of clarity for me around which industry classification levels are most suitable for these analyses, as discussed in Part 7 of the Database Exploration.

 Lastly, it would be useful to be able to pair the existing tables in this database with data that enables viewing these topics with a wider lens. For example, linking weekly payrolls to the Consumer Price Index (CPI) or the Gross Domestic Product (GDP) across different time periods would help standardize results against changes in cost of living and economic conditions, which could uncover some interesting insights.